In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from opym

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from opym

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs and calls the new validation/alignment function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    # --- NEW: Call the new processing function from opym ---
    # This function handles validation, alignment, and None assignment
    top_roi, bottom_roi = opym.process_rois_from_selector(
        mip_data,
        unaligned_rois,
        valid_threshold=1.0,
    )
    # --- END NEW ---

    if top_roi or bottom_roi:
        print("\nRun Cell 4 to visualize.")
    else:
        print("  Re-run Cell 2 to try again.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'mip_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from opym

from typing import cast

try:
    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs found. Please re-run Cell 2 and 3.")
    elif top_roi is None:
        print("ℹ️ Only Bottom ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know bottom_roi is not None here
        valid_bottom = cast(tuple[slice, slice], bottom_roi)
        opym.visualize_alignment(mip_data, valid_bottom, valid_bottom, vmin, vmax)
    elif bottom_roi is None:
        print("ℹ️ Only Top ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know top_roi is not None here
        valid_top = cast(tuple[slice, slice], top_roi)
        opym.visualize_alignment(mip_data, valid_top, valid_top, vmin, vmax)
    else:
        # Both are valid, show alignment
        opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)

    print("\nRun Cell 5 to save these ROIs to your log file.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # This is the central log file used by the CLI, as per the README.
    cli_log_file = Path("opm_roi_log.json")

    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs to save. Please re-run Cells 2 & 3.")
    else:
        opym.save_rois_to_log(
            log_file=cli_log_file,
            base_file=file_to_inspect,
            top_roi=top_roi,  # Can be None
            bottom_roi=bottom_roi,  # Can be None
        )
        print(f"✅ ROIs saved to {cli_log_file.name}")
        print("You can now use these ROIs for batch processing via the CLI,")
        print("or run Cell 6 to process this file immediately.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 6: Submit Cropping Job & Save Settings Sidecar ---

import importlib
import sys
import json
import re
from pathlib import Path
from typing import cast
import tifffile

import ipywidgets as widgets
from IPython.display import display

# --- 1. Import opym ---
try:
    import opym
    import opym.petakit 
    from opym.utils import OutputFormat
    from opym.roi_utils import _tuple_to_cli_string, _roi_to_tuple

    # Reload to ensure freshness
    importlib.reload(opym.petakit) 
    print("Successfully re-loaded opym package.")

    # --- 2. Detect Channels ---
    if 'file_to_inspect' not in globals():
        raise NameError("file_to_inspect is not defined. Run previous cells.")

    try:
        with tifffile.TiffFile(file_to_inspect) as tif:
            if hasattr(tif, "series") and len(tif.series) > 0:
                 shape = tif.series[0].shape
                 if len(shape) == 5: n_channels_in = shape[2]
                 elif len(shape) == 4: n_channels_in = shape[1]
                 else:
                     summary = tif.micromanager_metadata.get('Summary', {}) # type: ignore
                     n_channels_in = summary.get('Channels', 2)
            else: n_channels_in = 4
    except: n_channels_in = 4

    n_excitations = max(1, n_channels_in // 2) 
    print(f"Detected {n_channels_in} Input Channels -> {n_excitations} Excitation group(s).")

    # --- 3. Widgets ---
    output_format_widget = widgets.RadioButtons(
        options=[(f.value, f) for f in OutputFormat],
        description="Output Format:",
        value=OutputFormat.TIFF_SERIES,
    )
    rotate_widget = widgets.Checkbox(value=True, description="Rotate 90° (CCW)")

    channel_checks = {}
    ui_rows = [widgets.Label(f"Channels to Output ({n_channels_in*2} possible):")]

    for exc in range(n_excitations):
        base_id = exc * 4
        lbl = widgets.Label(f"--- Group {exc+1} ---")
        ui_rows.append(lbl)
        
        c_bot0 = widgets.Checkbox(value=True, description=f"C{base_id} (Bot, Cam 1)")
        c_top0 = widgets.Checkbox(value=True, description=f"C{base_id+1} (Top, Cam 1)")
        c_top1 = widgets.Checkbox(value=True, description=f"C{base_id+2} (Top, Cam 2)")
        c_bot1 = widgets.Checkbox(value=True, description=f"C{base_id+3} (Bot, Cam 2)")
        
        channel_checks[base_id]   = c_bot0
        channel_checks[base_id+1] = c_top0
        channel_checks[base_id+2] = c_top1
        channel_checks[base_id+3] = c_bot1
        
        ui_rows.append(widgets.HBox([c_bot0, c_top0]))
        ui_rows.append(widgets.HBox([c_top1, c_bot1]))

    run_button = widgets.Button(description="Submit & Save Settings", button_style="primary", icon="save")
    status_label = widgets.Label(value="Ready.")
    run_output = widgets.Output()

    display(widgets.VBox([output_format_widget, rotate_widget, widgets.VBox(ui_rows), widgets.HBox([run_button, status_label]), run_output]))

    # Global path holder
    processing_output_dir: Path | None = None

    # --- 4. Submit Function ---
    def on_run_button_clicked(b):
        global processing_output_dir

        with run_output:
            run_output.clear_output()
            try:
                # A. Gather Settings
                channels_to_output = sorted([cid for cid, cb in channel_checks.items() if cb.value])
                if not channels_to_output: print("❌ No channels selected."); return
                if top_roi is None or bottom_roi is None: print("❌ ROIs missing."); return

                fmt = cast(OutputFormat, output_format_widget.value).value
                rot = rotate_widget.value

                # B. Submit Job
                print("🚀 Submitting job to Petakit Queue...")
                job_path = opym.petakit.submit_remote_crop_job(
                    base_file=file_to_inspect,
                    top_roi=top_roi,
                    bottom_roi=bottom_roi,
                    channels=channels_to_output,
                    output_format=fmt,
                    rotate=rot
                )
                print(f"✅ Job Ticket Created: {job_path.name}")

                # C. Determine Output Dir & Save Sidecar
                base_name = file_to_inspect.name
                if base_name.lower().endswith(".ome.tif"): clean_name = base_name[:-8]
                elif base_name.lower().endswith(".tif"): clean_name = base_name[:-4]
                else: clean_name = file_to_inspect.stem

                processing_output_dir = file_to_inspect.parent / clean_name
                if not processing_output_dir.exists(): processing_output_dir.mkdir(parents=True)

                # --- SAVE SIDECAR ---
                sidecar_path = processing_output_dir / "petakit_settings.json"
                
                # Convert ROIs to clean strings "y1:y2,x1:x2"
                roi_top_str = _tuple_to_cli_string(_roi_to_tuple(top_roi))
                roi_bot_str = _tuple_to_cli_string(_roi_to_tuple(bottom_roi))
                
                settings = {
                    "source_file": str(file_to_inspect.name),
                    "rois": {
                        "top": roi_top_str,
                        "bottom": roi_bot_str
                    },
                    "channels": channels_to_output,
                    "rotate": rot,
                    "format": fmt,
                    "timestamp": "now" # You can add real time if needed
                }
                
                with open(sidecar_path, "w") as f:
                    json.dump(settings, f, indent=4)
                    
                print(f"💾 Settings Saved: {sidecar_path.name}")
                print(f"   Target Output Dir: {processing_output_dir.name}")
                
                # Monitor
                status_label.value = "⏳ Running..."
                opym.petakit.monitor_job_background(job_path, status_label)

            except Exception as e:
                import traceback; traceback.print_exc()

    run_button.on_click(on_run_button_clicked)

except ImportError:
    print("❌ ERROR: Could not import opym package.", file=sys.stderr)

In [ ]:
# --- Cell 7: Submit Deskew Job & Update Sidecar ---
import sys
import json
import importlib
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
import opym.petakit
import opym.metadata

# --- 1. SAFEGUARD: Ensure valid Path ---
# This fixes the Pylance "None" errors by guaranteeing 'target_dir' is a real Path
if 'processing_output_dir' not in globals() or processing_output_dir is None:
    print("⚠️ Output directory not defined. Please run Cell 6 first.")
    target_dir = None
else:
    target_dir = processing_output_dir

if target_dir is not None:
    # --- 2. Auto-Detect Z-Step from Metadata ---
    detected_z = 1.0
    meta_file = target_dir.parent / "AcqSettings.txt"
    if meta_file.exists():
        try: 
            detected_z = opym.metadata.parse_z_step(meta_file, 1.0)
            print(f"📄 Auto-detected Z-step from AcqSettings: {detected_z} µm")
        except: 
            print("⚠️ Could not parse AcqSettings. Using default 1.0 µm.")

    # --- 3. Widgets ---
    z_step_widget = widgets.FloatText(value=detected_z, description="Z Step (µm):", step=0.1)
    angle_widget  = widgets.FloatText(value=31.8, description="Angle (deg):", step=0.1)
    px_size_widget= widgets.FloatText(value=0.136, description="XY Pixel (µm):", step=0.001)
    
    run_deskew_btn = widgets.Button(description="Submit Deskew & Update JSON", button_style="success", icon="play")
    
    # NEW: Status label required for monitor_job_background
    deskew_status = widgets.Label(value="Ready to submit.")
    deskew_output = widgets.Output()

    display(widgets.VBox([
        widgets.HBox([z_step_widget, angle_widget, px_size_widget]),
        widgets.HBox([run_deskew_btn, deskew_status]),
        deskew_output
    ]))

    # --- 4. Submit Function ---
    def on_deskew_click(b):
        # Re-verify target inside button click for runtime safety
        if target_dir is None: return

        with deskew_output:
            deskew_output.clear_output()
            try:
                # A. Submit the Job
                print(f"🚀 Submitting Deskew Job for: {target_dir.name}...")
                
                ticket = opym.petakit.submit_remote_deskew_job(
                    input_target=target_dir, # Now guaranteed to be a Path
                    z_step_um=z_step_widget.value,
                    xy_pixel_size=px_size_widget.value,
                    sheet_angle_deg=angle_widget.value,
                    deskew=True,
                    rotate=True
                )
                print(f"✅ Ticket Created: {ticket.name}")

                # B. UPDATE the JSON Sidecar
                sidecar_path = target_dir / "petakit_settings.json"
                
                if sidecar_path.exists():
                    with open(sidecar_path, "r") as f:
                        settings = json.load(f)
                else:
                    settings = {} 

                # Append Deskew Settings
                settings["deskew"] = {
                    "z_step_um": z_step_widget.value,
                    "sheet_angle_deg": angle_widget.value,
                    "xy_pixel_size": px_size_widget.value
                }

                with open(sidecar_path, "w") as f:
                    json.dump(settings, f, indent=4)

                print(f"💾 Settings Updated: Added Deskew parameters to {sidecar_path.name}")
                
                # Monitor (Passing the required status label)
                deskew_status.value = "⏳ Job Running..."
                opym.petakit.monitor_job_background(ticket, deskew_status)

            except Exception as e:
                import traceback; traceback.print_exc()
                deskew_status.value = "❌ Error occurred."

    run_deskew_btn.on_click(on_deskew_click)

In [ ]:
# --- Cell 8: Load Final Processed Data ---
# This cell loads the final deskewed + rotated TIFF series
# from the 'DSR' directory created in Cell 7.

import sys
import importlib
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

# Import your package
import opym 
# Ensure dataloader is refreshed
importlib.reload(opym.dataloader)

# --- SAFEGUARD: Initialize globals if they don't exist ---
# This fixes the Pylance "UnboundVariable" errors
if "processing_output_dir" not in globals():
    processing_output_dir = None
if "viewer_params" not in globals():
    viewer_params = None

# --- Create Widgets ---
load_dsr_button = widgets.Button(
    description="Load Final DSR Data", 
    button_style="primary",
    icon="folder-open"
)
load_dsr_output = widgets.Output()

# --- Define the function to run on button click ---
def on_load_dsr_button_clicked(b):
    global viewer_params, processing_output_dir

    with load_dsr_output:
        load_dsr_output.clear_output()

        try:
            # 1. Resolve the Target Directory
            target_dsr = None
            
            # Scenario A: We have the variable from previous cells
            if processing_output_dir is not None:
                candidate = processing_output_dir / "DSR"
                if candidate.exists():
                    target_dsr = candidate
            
            # Scenario B: Variable missing, try to auto-detect
            if target_dsr is None:
                print("⚠️ 'processing_output_dir' not set. Searching for recent 'DSR' folders...")
                # Look for any folder ending in 'DSR' in the current directory
                found_dsrs = sorted(list(Path(".").glob("*/DSR")), key=lambda p: p.stat().st_mtime, reverse=True)
                
                if found_dsrs:
                    target_dsr = found_dsrs[0] # Pick the most recently modified one
                    print(f"   -> Found auto-detected folder: {target_dsr}")
                else:
                    print("❌ ERROR: Could not auto-detect any 'DSR' folders.")
                    print("   Please run Cell 6 & 7 first, or ensure your output folder exists.")
                    return

            print(f"--- Loading data from: {target_dsr} ---")

            # 2. Call the data loader
            (
                get_stack,
                t_min,
                t_max,
                c_min,
                c_max,
                z_max,
                y,
                x,
                base_name,
            ) = opym.load_tiff_series(target_dsr)

            # 3. Store parameters in the global variable
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)

            print(
                f"✅ Data loaded successfully.\n"
                f"   Shape: T={t_min}-{t_max}, Z={z_max + 1}, C={c_min}-{c_max}, Y={y}, X={x}\n"
                f"   Base name: {base_name}"
            )
            print("\nUsage: Run the viewer cell below to visualize.")

        except FileNotFoundError as e:
            print("\n❌ ERROR: Could not load data.")
            print(f"  Details: {e}")
            print("  Check if the 'DSR' folder is empty or contains incompatible files.")
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f"\n❌ An unexpected error occurred: {e}", file=sys.stderr)


# --- Link button and display UI ---
load_dsr_button.on_click(on_load_dsr_button_clicked)
display(widgets.VBox([load_dsr_button, load_dsr_output]))

In [ ]:
# --- Cell 9: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 10: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 11: Batch Process using "Gold Standard" Sidecar ---
import sys
import json
import time
from pathlib import Path
import opym.petakit
import opym.metadata

# ==============================================================================
# 📂 TEMPLATE SELECTION
# ==============================================================================
if 'processing_output_dir' in globals() and processing_output_dir is not None:
    TEMPLATE_FOLDER = processing_output_dir
else:
    # ⚠️ EDIT THIS PATH if restarting notebook
    TEMPLATE_FOLDER = Path("/path/to/your/processed/dhDF_cell1_MMStack_Pos0")

# ==============================================================================
# ⚙️ LOAD SETTINGS
# ==============================================================================
sidecar_file = TEMPLATE_FOLDER / "petakit_settings.json"

if not sidecar_file.exists():
    print(f"❌ ERROR: Sidecar file not found at:\n   {sidecar_file}")
    raise FileNotFoundError("Missing petakit_settings.json")

print(f"📖 Loading settings from: {sidecar_file.name}")
with open(sidecar_file, "r") as f:
    settings = json.load(f)

# Parse Crop Settings
channels = settings["channels"]
rotate   = settings["rotate"]
fmt      = settings["format"]

# Parse Deskew Settings (Defaults if missing)
ds_settings = settings.get("deskew", {})
TEMPLATE_ANGLE = ds_settings.get("sheet_angle_deg", 31.8)
TEMPLATE_PIXEL = ds_settings.get("xy_pixel_size", 0.136)
TEMPLATE_Z     = ds_settings.get("z_step_um", 1.0)

# Parse ROIs
def parse_roi_str(s):
    parts = s.split(',')
    y = [int(v) for v in parts[0].split(':')]
    x = [int(v) for v in parts[1].split(':')]
    return (slice(y[0], y[1]), slice(x[0], x[1]))

top_roi_slice = parse_roi_str(settings["rois"]["top"])
bot_roi_slice = parse_roi_str(settings["rois"]["bottom"])

print(f"   ✅ Config Loaded:")
print(f"      - ROIs:      Top={settings['rois']['top']}")
print(f"      - Channels:  {channels}")
print(f"      - Deskew:    Angle={TEMPLATE_ANGLE}°, Pixel={TEMPLATE_PIXEL}µm")

# ==============================================================================
# 🚀 BATCH EXECUTION
# ==============================================================================
data_root = TEMPLATE_FOLDER.parent
print(f"\n📂 Scanning directory: {data_root}")

all_files = sorted(list(data_root.glob("**/*_MMStack_Pos0.ome.tif")))
files_to_process = [f for f in all_files if f.name != settings.get("source_file", "")]

print(f"🔎 Found {len(all_files)} total files.")
print(f"🚀 Queueing {len(files_to_process)} remaining files...")

submitted_jobs = []

for i, file_path in enumerate(files_to_process):
    print(f"\n[{i+1}/{len(files_to_process)}] Processing: {file_path.name}")
    
    try:
        # A. Submit CROP Job
        crop_ticket = opym.petakit.submit_remote_crop_job(
            base_file=file_path,
            top_roi=top_roi_slice,
            bottom_roi=bot_roi_slice,
            channels=channels,
            output_format=fmt,
            rotate=rotate
        )
        
        # B. Calculate Target Dir
        base_name = file_path.name
        if base_name.lower().endswith(".ome.tif"): clean_name = base_name[:-8]
        elif base_name.lower().endswith(".tif"): clean_name = base_name[:-4]
        else: clean_name = file_path.stem
        target_dir = file_path.parent / clean_name
        
        # C. Submit DESKEW Job
        # Smart Z-Step: Try to read file metadata, fallback to Template value
        z_step = TEMPLATE_Z
        meta_file = file_path.parent / "AcqSettings.txt"
        if meta_file.exists():
             try: z_step = opym.metadata.parse_z_step(meta_file, TEMPLATE_Z)
             except: pass

        deskew_ticket = opym.petakit.submit_remote_deskew_job(
            input_target=target_dir,
            z_step_um=z_step,                 # File-specific or Template fallback
            xy_pixel_size=TEMPLATE_PIXEL,     # From Template
            sheet_angle_deg=TEMPLATE_ANGLE,   # From Template
            deskew=True,
            rotate=True
        )
        
        submitted_jobs.append((file_path.name, crop_ticket, deskew_ticket))
        print(f"   ✅ Submitted Crop & Deskew tickets.")

    except Exception as e:
        print(f"   ❌ Failed to submit {file_path.name}: {e}")

# --- MONITORING ---
if submitted_jobs:
    print(f"\n⏳ Monitoring {len(submitted_jobs)*2} Jobs...")
    try:
        while True:
            all_done = True
            pending = 0
            for name, crop_t, deskew_t in submitted_jobs:
                base = deskew_t.parent.parent
                if (base / "queue" / deskew_t.name).exists(): pending += 1; all_done = False
                elif (base / "queue" / crop_t.name).exists(): pending += 1; all_done = False
            
            sys.stdout.write(f"\r📊 Queue Status: {pending} jobs pending...   ")
            sys.stdout.flush()
            if all_done: break
            time.sleep(2)
        print("\n\n✅ All batch jobs completed!")
    except KeyboardInterrupt:
        print("\n🛑 Monitoring stopped.")